In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/models/projgrad')
!pip install lmfit


     |████████████████████████████████| 307kB 6.3MB/s 
     |████████████████████████████████| 61kB 4.9MB/s 
     |████████████████████████████████| 256kB 8.1MB/s 
  Created wheel for lmfit: filename=lmfit-1.0.2-cp37-none-any.whl size=85265 sha256=5c318511965ad69d72abc8b1343af345b8ef0ce86a2ab6404012a4bcc77b7848
  Stored in directory: /root/.cache/pip/wheels/d4/5e/db/d9f5987d9f15bf910feb5e0274ac601d5f2161dc78aedfacfa
  Created wheel for asteval: filename=asteval-0.9.23-cp37-none-any.whl size=17419 sha256=4eb677fe3d84346eef5beeec20d400cfdb9c1aa522e246f0d247af89ea491ada
  Stored in directory: /root/.cache/pip/wheels/56/d6/43/4b5d8a9fd98b976031768673658c8b8d08dc9dcc0fa1907ad0
Successfully built lmfit asteval


In [ ]:
from lmfit import Parameters, Parameter
from scipy.integrate import odeint
# from liboptpy.benchmark_algorithms.DDPG import get_actor
from liboptpy.lib.mechanism_model import mechanism
from liboptpy.policy import Policy, PolicyLab, PolicyDDPG
from liboptpy.simulator.BaseSimulator import BaseSimulator
from liboptpy.gradients.bayesian_network import bayesian_network, bayesian_network_posterior
from liboptpy.gradients.BN_gradient import BN_gradient
import liboptpy.constr_solvers._proj_gd as cs
import liboptpy.step_size as ss
import numpy as np
import pandas as pd
from liboptpy.simulator.bio_env import fermentation_env, fermentation_env2

In [ ]:

params = Parameters()

params.add('alpha_L', value=0.127275, min=0, max=0.5)
params.add('c_max', value=130.901733, min=100, max=250)
params.add('K_iN', value=0.12294103, min=0, max=5.064)
params.add('K_iS', value=612.178130, min=300, max=700)
params.add('K_iX', value=59.9737695, min=20, max=100)
params.add('K_N', value=0.02000201, min=0, max=1)
params.add('K_O', value=0.33085322, min=0, max=10)
params.add('K_S', value=0.0430429, min=0, max=5)
params.add('K_SL', value=0.02165744, min=0, max=1)
params.add('m_s', value=0.02252332, min=0, max=2)
params.add('r_L', value=0.47917813, min=0, max=1)
params.add('V_evap', value=2.6 * 1e-03, min=0, max=10)
params.add('Y_cs', value=0.682572, min=0, max=4)
params.add('Y_ls', value=0.3574429, min=0, max=2)
params.add('Y_xn', value=10.0, min=0, max=15)
params.add('Y_xs', value=0.2385559, min=0, max=1)
params.add('beta_LC_max', value=0.14255192, min=0, max=5)
params.add('mu_max', value=0.3844627, min=0, max=2)

import matplotlib.pyplot as plt
from tqdm import tqdm

# build bayesian network
beta_gibbs = pd.read_csv('./liboptpy/BN/beta_s5a1-R100-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
v2_gibbs = pd.read_csv('./liboptpy/BN/v2_s5a1-R100-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None,
                        dtype=np.float64)
the_R = pd.read_csv('./liboptpy/BN/the_R_s5a1-R100-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
tau2_R = pd.read_csv('./liboptpy/BN/tau2_R_s5a1-R100-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
kap_R = pd.read_csv('./liboptpy/BN/kap_R_s5a1-R100-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
lam_R = pd.read_csv('./liboptpy/BN/lam_R_s5a1-R100-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
mu = pd.read_csv('./liboptpy/BN/mu_s5a1-R100-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
sd = pd.read_csv('./liboptpy/BN/sd_s5a1-R100-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)

bn_post = bayesian_network_posterior(the_R.to_numpy(), tau2_R.to_numpy(), kap_R.to_numpy(), lam_R.to_numpy(),
                                      mu.to_numpy().flatten(), 5, 1, 36, beta_gibbs, v2_gibbs)
p_beta, p_v2, mu = bn_post.posterior_sample(5000, useFixed=False)
beta = np.apply_along_axis(np.mean, 2, p_beta)
v2 = np.apply_along_axis(np.mean, 1, p_v2)
bn = bayesian_network(mu, beta, v2, 1, 5, 36, True, mu, sd.to_numpy().flatten())

feed_profile = [0.0000000000,
                0.0003624999,
                0.0048485723,
                0.0119220315,
                0.0102440275,
                0.0086513113,
                0.0061447560,
                0.0039552809,
                0.0048855832,
                0.0020873176,
                0.0017531639,
                0.0015830354,
                0.0010269718,
                0.0004710181]
feed_profile = [0.0000000000, 0.0003131863, 0.0041663280, 0.0098325692, 0.0087833820, 0.0077948404, 0.0056611497, 0.0038797959, 0.0041133889, 0.0019123248, 0.0015901411, 0.0015340615, 0.0011494357, 0.0004164647]


# feed_profile = [0, 4.440000e-19, 0.002515011, 0.004502748, 0.005973753, 0.007297723, 0.006291863,  5.175258e-03, 0.0025989402, 0.001611073, 0.001167248, 0.001135921, 0.0010017584, 0.0001669597]
# feed_profile = [0.000000e+00,  0.000000e+00,  2.801345e-03,  6.500561e-03,  7.838870e-03,  7.379049e-03,  5.426248e-03, -4.236836e-05,
#  1.575191e-02,  1.355505e-03, -1.190099e-03,  1.957627e-03,  9.760737e-04,  1.626789e-04]
real_measurement = [0.0, 5.0, 10.0, 23.0, 28.0, 34.0, 47.5, 55.0, 72.0, 80.0, 95.0, 102.0, 120.0, 140.0]


class bioprocess_simulator(BaseSimulator):
    def __init__(self, bn, time_measurement, policy, real_measurement):
        self.bn = bn
        # self.feed = [0] *  bn.n_time
        self.oxygen = [0] * 14 # bn.n_time
        self.current_time = -1
        self.cur_action = None
        self.policy = policy
        self.feed = {}
        self.feed_every_second = []
        self.time_measurement = time_measurement
        self.real_measurement = real_measurement

    def feed_rate(self, t, xs):
        state_index = [0, 1, 3, 4] if self.bn.num_state == 4 else range(5)
        if self.bn.num_state == 5 and self.bn.num_action == 1:
            state_index = [0, 1, 3, 4, 5]
        index = 0
        time_w = -1
        for i, time in enumerate(self.time_measurement):
            if time - t > 0:
                index = i - 1
                time_w = time
                break

        ### model-free RL
        state = xs
        if self.policy.policy_algo == 'DDPG':
            a_t = self.policy.next_action(state, index)
            print(a_t[0], index)
            return a_t[0]

        ### human policy

        if not self.policy.scale_needed:
            a_t = self.policy.next_action([], t)
            # self.feed[self.current_time] = a_t
            return a_t

        ### BN-MDP
        state = (state[state_index] - self.bn.mu[(self.bn.n_factor * (index + 1) - self.bn.num_state):(
                    self.bn.n_factor * (index + 1))]) / \
                self.bn.sample_sd[(self.bn.n_factor * (index + 1) - self.bn.num_state):(self.bn.n_factor * (index + 1))]
        # print(index)

        self.current_time = index
        if index in self.feed.keys():
            return self.feed[self.current_time]

        if self.policy.scale_needed:
            a_t = self.policy.next_action(state, index)
            action = self.bn.rescale_action(a_t, index)
            if action[0] < 0:
                action[0] = 0
            if action[0] > 0.02:
                action[0] = 0.02
            if len(action) > 1 and action[1] < 0:
                action[1] = 0
            if len(action) > 1 and action[1] > 100:
                action[1] = 100
            # self.feed[self.current_time] = action[0]
            return action[0]


    def dissolve_oxygen(self, t):
        oxygen_data = [98.58750, 71.34750, 43.57285, 28.51669, 24.61062, 24.58139, 28.96252, 30.63478, 24.34490,
                       29.33842, 31.89646, 34.51994, 36.03211, 36.16354]

        index = 0
        time_w = -1
        for i, time in enumerate(self.real_measurement):
            if time - t > 0:
                index = i - 1
                time_w = time
                break
        # index = 1
        self.oxygen[index] = oxygen_data[index]
        return oxygen_data[index]

    def f(self, xs, t, ps):
        """bioreactor model."""
        alpha_L = ps['alpha_L'].value
        c_max = ps['c_max'].value
        K_iN = ps['K_iN'].value
        K_iS = ps['K_iS'].value
        K_iX = ps['K_iX'].value
        K_N = ps['K_N'].value
        K_O = ps['K_O'].value
        K_S = ps['K_S'].value
        K_SL = ps['K_SL'].value
        m_s = ps['m_s'].value
        r_L = ps['r_L'].value
        #    V_evap = ps['V_evap'].value
        V_evap = ps['V_evap'].value
        Y_cs = ps['Y_cs'].value
        Y_ls = ps['Y_ls'].value
        Y_xn = ps['Y_xn'].value
        Y_xs = ps['Y_xs'].value

        beta_LC_max = ps['beta_LC_max'].value
        mu_max = ps['mu_max'].value
        # oil concentration in oil feed
        S_F = 917

        X_f, C, L, S, N, V = xs
        t_index = t/ (140/(self.bn.n_time-1))
        if int(t_index) in self.feed.keys():
            F_S, O = self.feed[int(t_index)], self.dissolve_oxygen(t)
        elif self.bn.num_action == 1:
            F_S, O = self.feed_rate(t, xs), self.dissolve_oxygen(t)
            self.feed_every_second.append([t, F_S])
            self.feed[int(t_index)] = F_S
        elif self.bn.num_action == 2:
            F_S, O = self.feed_rate(t, xs)
            self.feed_every_second.append([t, F_S])
            self.feed[int(t_index)] = F_S

        beta_LC = K_iN / (K_iN + N) * S / (S + K_S) * K_iS / (K_iS + S) * O / (K_O + O) * K_iX / (K_iX + X_f) * (
                    1 - C / c_max) * beta_LC_max
        q_C = 2 * (1 - r_L) * beta_LC
        beta_L = r_L * beta_LC - K_SL * L / (L + X_f) * O / (O + K_O)
        mu = mu_max * S / (S + K_S) * K_iS / (K_iS + S) * N / (K_N + N) * O / (K_O + O) / (1 + X_f / K_iX)
        q_S = 1 / Y_xs * mu + O / (O + K_O) * S / (K_S + S) * m_s + 1 / Y_cs * q_C + 1 / Y_ls * beta_L
        F_B = V / 1000 * (7.14 / Y_xn * mu * X_f + 1.59 * q_C * X_f)
        D = (F_B + F_S) / V
        # print("{0}:{1}:{2}:{3}".format(t,mu,q_C, F_S))
        dX_f = mu * X_f - (D - V_evap / V) * X_f
        dC = q_C * X_f - (D - V_evap / V) * C
        dL = (alpha_L * mu + beta_L) * X_f - (D - V_evap / V) * L
        dS = - (q_S * X_f - F_S / V * S_F + (D - V_evap / V) * S)
        dN = - (1 / Y_xn * mu * X_f + (D - V_evap / V) * N)
        dV = F_B + F_S - V_evap
        return [dX_f, dC, dL, dS, dN, dV]

    def g(self, t, x0, ps):
        """
        Solution to the ODE x'(t) = f(t,x,k) with initial condition x(0) = x0
        """
        x = odeint(self.f, x0, t, args=(ps,), rtol=1.49012e-106, mxstep=200000)
        return x
      
cumulative_rewards = []
final_titer = []
total_feeding = []
feed_profiles = []
conv_trajectories = []
process_model = mechanism()
env = fermentation_env2(process_model)
patient = 15
window = 15
max_iter = 100
algorithm = 'PGA'
fixedBN = False
for id in range(30):
    feed_profile_ddpg = []
    np.random.seed(128 + id)

    time_interval = 140 / (bn.n_time - 1)
    time_measurement = [time_interval * i for i in range(bn.n_time)]
    flag_estimate_reward_fun = False
    m = [0] * (bn.n_time - 1) + [-15]  # $/g
    b = np.transpose(np.array([[-0.1 * 1000 * time_interval] * (bn.n_time - 1)]))  # $/g
    c = np.zeros(shape=(bn.n_time, bn.num_state))
    c[bn.n_time - 1, 1] = 1.28739  # product revenue
    c[bn.n_time - 1, 0] = 0  # purification cost
    c[bn.n_time - 1, 2] = 0  # purification cost

    if algorithm == "PSGA":
        gradient = BN_gradient(m, b, c, bn, time_measurement, flag_estimate_reward_fun)
        # 1972 # 1983 (0.2)
        theta = np.random.uniform(low=0., high=0.3, size=(bn.n_time - 1, bn.num_state, bn.num_action))
        grad = lambda x: - gradient.grad_f(x)
        f = lambda x: - gradient.func(x)


        def projection(y):
            theta = y.reshape((bn.n_time - 1, bn.num_state, bn.num_action))
            theta[:, 0, :] = np.clip(theta[:, 0, :], 0, 0.6)
            theta[:, 1, :] = np.clip(theta[:, 0, :], 0, 0.6)
            theta[0, 2, :] = np.clip(theta[0, 2, :], -0.2, 0.2)
            theta[:, 2, :] = np.clip(theta[:, 2, :], -0.4, 0.4)
            theta[:, 3, :] = np.clip(theta[:, 3, :], -0.2, 0.04)
            theta[:, 4, :] = np.clip(theta[:, 4, :], -1.4, 1)
            return theta.flatten()


        process_model = mechanism()
        bio_env = fermentation_env(process_model, bn, time_measurement, real_measurement, params)
        if not fixedBN:
            methods = {"PSGA": cs.ProjectedSGD(f,
                                                grad,
                                                projection,
                                                # ss.InvSqrootIterStepSize(),
                                                ss.Backtracking(rule_type="Armijo", rho=0.5, beta=0.1, init_alpha=0.1),
                                                bn_post,
                                                sd.to_numpy().flatten(),
                                                bio_env,
                                                batch_size=64,
                                                B=256,
                                                patient=patient,
                                                window=window)
                        }
        else:
            methods = {"PSGA": cs.ProjectedBNSGD(f,
                                                grad,
                                                projection,
                                                # ss.InvSqrootIterStepSize(),
                                                ss.Backtracking(rule_type="Armijo", rho=0.5, beta=0.1, init_alpha=0.1),
                                                bn_post,
                                                sd.to_numpy().flatten(),
                                                bio_env,
                                                batch_size=64,
                                                B=256,
                                                patient=patient,
                                                window=window)
                        }


        tol = 1e-25
        for m_name in methods:
            print("\t", m_name)
            x = methods[m_name].solve(x0=theta.flatten(), max_iter=max_iter, tol=tol, disp=2)
        # gradient.states
        # [f(x) for x in methods[m_name].get_convergence()]

        theta = x.reshape((bn.n_time - 1, bn.num_state, bn.num_action))
        policy = Policy(theta, True)
    else:
        gradient = BN_gradient(m, b, c, bn, time_measurement, flag_estimate_reward_fun)
        theta = np.random.uniform(low=0., high=0.3, size=(bn.n_time - 1, bn.num_state, bn.num_action))
        grad = lambda x: - gradient.grad_f(x)
        f = lambda x: - gradient.func(x)


        def projection(y):
            theta = y.reshape((bn.n_time - 1, bn.num_state, bn.num_action))
            theta[:, 0, :] = np.clip(theta[:, 0, :], 0, 0.6)
            theta[:, 1, :] = np.clip(theta[:, 0, :], 0, 0.6)
            theta[0, 2, :] = np.clip(theta[0, 2, :], -0.2, 0.2)
            theta[:, 2, :] = np.clip(theta[:, 2, :], -0.4, 0.4)
            theta[:, 3, :] = np.clip(theta[:, 3, :], -0.2, 0.04)
            theta[:, 4, :] = np.clip(theta[:, 4, :], -1.4, 1)
            return theta.flatten()

        process_model = mechanism()
        bio_env = fermentation_env(process_model, bn, time_measurement, real_measurement, params)
        if not fixedBN:
            methods = {"PGA": cs.ProjectedGDWithPosterior(f,
                                                      grad,
                                                      projection,
                                                      ss.Backtracking(rule_type="Armijo", rho=0.5, beta=0.1,
                                                                      init_alpha=0.1),
                                                      bn_post,
                                                      bio_env,
                                                      sd.to_numpy().flatten(),
                                                      256,
                                                      patient,
                                                      window)
                    }
        else:
            methods = {"PGA": cs.ProjectedBNPG(f,
                                                grad,
                                                projection,
                                                ss.Backtracking(rule_type="Armijo", rho=0.5, beta=0.1,
                                                              init_alpha=0.1),
                                                bn_post,
                                                bio_env,
                                                sd.to_numpy().flatten(),
                                                256,
                                                patient,
                                                window)
                        }

        tol = -1000 # 1e-25
        for m_name in methods:
            print("\t", m_name)
            x = methods[m_name].solve(x0=theta.flatten(), max_iter=max_iter, tol=tol, disp=2)
        # gradient.states
        # [f(x) for x in methods[m_name].get_convergence()]

        theta = x.reshape((bn.n_time - 1, bn.num_state, bn.num_action))
        policy = Policy(theta, True)
    '''bn
    simulate new policy and existing policy with true model (ODEs)
    '''
    B = 50
    x0 = bn.initial_state_full
    # + np.abs(
    #   np.random.normal(0, np.array(bn.initial_state_full) / 10 + 0.01))
    # time_measurement[-1] = 135.9
    simulator = bioprocess_simulator(bn, time_measurement, policy, real_measurement)
    state = simulator.g(time_measurement, x0, params)
    # feed_BN = pd.DataFrame(simulator.feed_every_second, columns=['time', 'feed_rate'], dtype="float64").drop_duplicates()
    # feed_BN = feed_BN.sort_values(by='time')
    feed_profile_ddpg = list(simulator.feed.values())
    total_feed_MDP = np.sum(feed_profile_ddpg)

    # simulator = bioprocess_simulator(bn, time_measurement, policy, real_measurement)
    # state = simulator.g(time_measurement, x0, params)
    # feed_BN = pd.DataFrame(simulator.feed_every_second, columns=['time', 'feed_rate'], dtype="float64").drop_duplicates()
    # feed_BN = feed_BN.sort_values(by='time')
    # feed_profile_ddpg = list(simulator.feed.values())
    # total_feed_MDP = np.sum(feed_profile_ddpg)
    conv_trajectories.append(methods[m_name].convergence_reward)
    # for i in range(feed_BN.shape[0]-1):
    #     total_feed_MDP += (feed_BN.iloc[i+1][0] - feed_BN.iloc[i][0]) * (feed_BN.iloc[i][1])

    final_titer.append(state[-1,1])
    feed_profiles.append(list(simulator.feed.values()))
    total_feeding.append(total_feed_MDP * (140 / (bn.n_time - 1)))
    final_reward = -0.13363 * 1000 * total_feed_MDP + 1.28739 * state[-1, 1]
    cumulative_rewards.append(final_reward)
    print('--------------------------------------------------')
    print(id)
    print("mean final titer = {}".format(state[-1,1]))
    print("mean reward = {}".format(final_reward))

# cumulative_rewards = [total_feeding[i] / 4 * -0.13363 * 1000 + 1.28739 * final_titer[i] for i in range(30)]


print(np.mean(final_titer), np.std(final_titer))
print(np.mean(cumulative_rewards), np.std(cumulative_rewards))
np.save('./BN-MDP-outputs/{}-fixed{}-final_titer-R100-sigma10.npy'.format(algorithm, fixedBN), final_titer)
np.save('./BN-MDP-outputs/{}-fixed{}-feed_profiles-R100-sigma10.npy'.format(algorithm, fixedBN), feed_profiles)
np.save('./BN-MDP-outputs/{}-fixed{}-total_feeding-R100-sigma10.npy'.format(algorithm, fixedBN), total_feeding)
np.save('./BN-MDP-outputs/{}-fixed{}-cumulative_rewards-R100-sigma10.npy'.format(algorithm, fixedBN), cumulative_rewards)

	 PGA
Iteration 1/100
Current function val = 114.74971920725369
Current reward = 5.329492096810526
Difference in function values = -173.28319513983973
Difference in argument = 5.119207902461929
Iteration 2/100
Current function val = -62.65569592364658
Current reward = -96.14390572938609
Difference in function values = 427.45163218797137
Difference in argument = 6.025685722077614
Iteration 3/100
Current function val = 10.297851942518236
Current reward = 59.0634369522615
Difference in function values = -22.638921484445007
Difference in argument = 12.587759491663455
Iteration 4/100
Current function val = 190.1597080671384
Current reward = 59.132557536442235
Difference in function values = -105.09507903735394
Difference in argument = 0.004607569771687906
Iteration 5/100
Current function val = 22.003546156025237
Current reward = 58.8188495993805
Difference in function values = 157.81261259565002
Difference in argument = 3.052832478589164
Iteration 6/100
Current function val = 158.7517881907